# Prompting

En este notebook vemos la evolución del IAA con distintas técnicas de Prompting.

Vamos a usar ChatGPT, el API-KEY se puede configurar en [Platform](https://platform.openai.com/settings/organization/api-keys).

In [ ]:
from openai import OpenAI
import krippendorff
import time
import os
import polars as pl

from dotenv import load_dotenv
load_dotenv()

import matplotlib.pyplot as plt
import pandas as pd

from IAA import (
    matriz_kripp, 
    calcula_alpha_kripp
)

### Cargamos los Dataset

In [245]:
dataset1 = pd.read_excel('data/dataset_1.xlsx', sheet_name='Dataset_1_Compartidos')
dataset2 = pd.read_excel('data/dataset_2.xlsx', sheet_name='Dataset_2_Compartidos')
dataset3 = pd.read_excel('data/dataset_3.xlsx', sheet_name='Sheet1')

In [246]:
X = dataset3[['nroV', 'idF', 'País', 'Día', 'Mes', 'Año', 'Medio', 'Título','Comentario']]
Y = dataset3['Calificación']

In [247]:
from sklearn.model_selection import train_test_split

X_train_vali, X_held_out, Y_train_vali, Y_held_out = train_test_split(X, Y, test_size=0.11, random_state=10)

X_train, X_vali, Y_train, Y_vali = train_test_split(X_train_vali, Y_train_vali, test_size=0.13, random_state=10)

Y_train.shape, Y_vali.shape

((276,), (42,))

## One shot learning

In [251]:
casos = ""
for i, row in X_vali.iterrows():
    data = row.to_dict()
    comentario = row['Comentario'].replace("\n", " ").strip()
    casos += f"> Id: {i} | Año: {row['Año']} | Título: '{row['Título'].strip()}' | Comentario: '{comentario}'\n"

print(casos)

> Id: 188 | Año: 2022 | Título: 'Benet se reúne con Putin en Moscú para mediar en la guerra de Ucrania' | Comentario: 'No mas OTAN, no mas apoyo de la OTAN al genocidio Israeli sobre el pueblo palestino....'
> Id: 149 | Año: 2022 | Título: '“No conocés Helueni, no conocés el Once”. El local de comida judía donde compraba Tato Bores y Molda' | Comentario: 'Habla de comida judía pero vienen de siria'
> Id: 129 | Año: 2021 | Título: 'Opinión | Cosas que hay que saber antes de juzgar a Israel' | Comentario: 'Según la señora que escribe los judios son unos santos o una carmelitas descalzas. Es una falta de respeto a la inteligencia este artículo. Israel es complice de hamas como lo fue del estado islamico al cual condeno muy poco.'
> Id: 250 | Año: 2021 | Título: 'Travesía histórica: cómo 23 judíos expulsados de Brasil ayudaron a fundar Nueva York' | Comentario: 'nadie los quiere. los rajon de todos lados. porque sera?'
> Id: 73 | Año: 2024 | Título: 'Docente de curso postergado en Udelar: 

In [ ]:
def calcula_iaa(prompt, llm):
    """ """
    ids = []
    clases = []

    for line in open(f'resultados/{prompt}/{llm}.txt').readlines():
        row = line.strip().split(" ")
        id = row[1]
        clase = row[-1].replace("\n",'')

        ids.append(int(id))
        clases.append(clase)

    preds = pd.DataFrame({"id": ids, 'Predicción': clases})   
    preds = pl.from_pandas(preds)
    y_vali = pd.DataFrame(Y_vali)
    y_vali['id'] = Y_vali.index
    y_vali = pl.from_pandas(y_vali)

    preds_and_real = y_vali.join(preds, on='id')
    kripp_matrix = preds_and_real.drop('id').to_pandas().values.astype(str).T
    alpha = calcula_alpha_kripp(kripp_matrix)

    return alpha

In [ ]:
def get_iaa_llm_table(prompt):
    """ """
    df = pd.DataFrame({
        'IAA_QWEN': [calcula_iaa(prompt=prompt, llm='qwen')],
        'IAA_GEMINI_PRO': [calcula_iaa(prompt=prompt, llm='gemini_pro')],
        'IAA_CLAUDE': [calcula_iaa(prompt=prompt, llm='claude')]
    })
    df = df.T
    df.columns = ['Inter-Annotator Agreement']

    return df

## Few shot learning

In [ ]:
train_ds = X_train.join(Y_train)

def armar_prompt(contexto, pedido):
    prompt = contexto
    for _, row in train_ds[['Título', 'Comentario', 'Calificación']].iterrows():
        data = row.to_dict()
        prompt += f"> Id: {row.index} | Título: '{row['Título'].strip()}' \
            | Comentario: '{row['Comentario'].strip()}' \
            | Calificación: '{row['Calificación'].strip()}'\n"
    prompt = "{pedido}"
    return prompt

